In [2]:
# import yfinance as yf
# import pandas as pd
# import time

# # List of crypto-related tickers
# tickers = [
#     "MSTR",   # MicroStrategy
#     "GLXY.TO",# Galaxy Digital (Toronto-listed)
#     "3350.T", # Metaplanet Inc (Tokyo Stock Exchange)
#     "COIN",   # Coinbase
#     "RIOT",   # Riot Platforms
#     "MARA",   # Marathon Digital
#     "CLSK",   # CleanSpark
#     "WULF",   # TeraWulf
#     "CORZ",   # Core Scientific
#     "BTDR"    # Bitdeer Technologies (NASDAQ) – you can swap with HIVE.V if preferred
# ]

# # Dictionary to store DataFrames
# price_data = {}

# for ticker in tickers:
#     print(f"Downloading {ticker}...")
#     ticker_obj = yf.Ticker(ticker)
#     try:
#         historyPrices = ticker_obj.history(
#             period="5y",
#             interval="1d",
#             threads=False  # Avoid multi-threaded requests which trigger rate limit
#         )
#         price_data[ticker] = historyPrices
#         print(f"{ticker} downloaded: {len(historyPrices)} rows")
#     except Exception as e:
#         print(f"Error downloading {ticker}: {e}")
#     time.sleep(5)  # Pause 5 seconds between requests to prevent rate limit

# Optionally: Combine into one Excel file with a sheet per ticker
# with pd.ExcelWriter("crypto_stocks_5y.xlsx") as writer:
#     for ticker, df in price_data.items():
#         df.to_excel(writer, sheet_name=ticker)



In [3]:
import pandas_datareader as pdr

import pandas as pd
import numpy as np

from datetime import datetime
import datetime

import matplotlib.pyplot as plt

from pathlib import Path

In [4]:
# Define the start and end dates for the data retrieval
start_date = '2020-01-01'
end_date = '2025-08-01'
#end_date

# FRED Macroeconomic Indicators

In [5]:
def get_fred_macro_data(fred_series, start_date, end_date):
    """
    Fetch FRED macroeconomic data, compute YoY and QoQ changes,
    and return a single merged DataFrame resampled to quarter-end.

    Parameters:
    - fred_series: dict of {friendly_name: FRED_series_code}
    - start_date, end_date: date strings in 'YYYY-MM-DD' format

    Returns:
    - macro_df: DataFrame with macro features per quarter
    """
    fred_data = {}

    for name, code in fred_series.items():
        try:
            df = pdr.DataReader(code, "fred", start=start_date, end=end_date)
            if df.empty:
                print(f"[FRED] No data for {name} ({code}), skipping.")
                continue
            # YoY and QoQ percentage changes
            df[name + '_yoy'] = df[code].pct_change(4)
            df[name + '_qoq'] = df[code].pct_change(1)

            # Rename column to friendly name
            df = df.rename(columns={code: name})

            fred_data[name] = df

        except Exception as e:
            print(f"[FRED] Error for {name} ({code}): {e}")
            continue

    if not fred_data:
        print("No FRED data retrieved.")
        return pd.DataFrame()

    # Merge and resample to quarter-end
    macro_df = pd.concat(fred_data.values(), axis=1).resample('QE').last()
    

    # Extract year and quarter for joining with other datasets
    macro_df = macro_df.reset_index()
    macro_df['year'] = macro_df['DATE'].dt.year
    macro_df['quarter'] = macro_df['DATE'].dt.quarter
    macro_df = macro_df.rename(columns={'DATE': 'date'})
    return macro_df

In [6]:
fred_series = {
    'gdp_us': 'GDPC1',
    'cpi_us': 'CPIAUCSL',
    'unemployment_us': 'UNRATE',
    'interest_us': 'FEDFUNDS',
    'gdp_de': 'CLVMNACSCAB1GQDE',
    'cpi_de': 'DEUCPIALLMINMEI',
    'interest_eu': 'IRLTLT01EZM156N'
}

In [7]:
macro_df = get_fred_macro_data(fred_series, start_date, end_date)
macro_df.tail()

,date,gdp_us,gdp_us_yoy,gdp_us_qoq,cpi_us,cpi_us_yoy,cpi_us_qoq,unemployment_us,unemployment_us_yoy,unemployment_us_qoq,...,gdp_de_yoy,gdp_de_qoq,cpi_de,cpi_de_yoy,cpi_de_qoq,interest_eu,interest_eu_yoy,interest_eu_qoq,year,quarter
18,2024-09-30,23400.294,0.027188,0.007595,314.851,0.005464,0.002292,4.1,0.02500,-0.02381,...,-0.006217,0.000192,126.1978,0.003353,0.000000,2.834165,-0.092048,-0.024690,2024,3
19,2024-12-31,23542.349,0.025337,0.006071,317.603,0.011053,0.003647,4.1,-0.02381,-0.02381,...,-0.001648,0.001822,127.0412,0.006683,0.005004,2.812343,-0.032200,-0.039871,2024,4
20,2025-03-31,23512.717,0.019918,-0.001259,319.615,0.010005,-0.000500,4.2,0.00000,0.02439,...,0.002487,0.003062,127.7792,0.010843,0.003311,3.313772,0.131316,0.108784,2025,1
21,2025-06-30,23703.782,0.020663,0.008126,321.500,0.005394,0.002870,4.1,0.00000,-0.02381,...,0.002301,-0.002767,NaN,NaN,NaN,3.084761,0.032157,0.004483,2025,2
22,2025-09-30,NaN,NaN,NaN,323.364,0.009500,0.003825,4.3,0.02381,0.02381,...,NaN,NaN,NaN,NaN,NaN,3.173563,-0.042311,0.028787,2025,3


In [8]:
macro_df.rename(columns={'date': 'quarter_end'}, inplace=True)

In [9]:
df_crypto = pd.read_csv('multi_crypto.csv', parse_dates=['timestamp'])

In [10]:
df_crypto.rename(columns={'timestamp': 'date'}, inplace=True)

In [11]:
#df_crypto.drop(columns=['open', 'high', 'low'], inplace=True)

In [12]:
# Data preprocessing

In [15]:
def get_price_features(df):
        df = df.copy()

        #drop unused columns
        df.columns = df.columns.str.lower()
        df.drop(columns=['open', 'high', 'low'], inplace=True)

        # 1. Log of Volume (avoid log(0) by replacing 0 with NaN)
        df['ln_volume'] = np.log(df['volume'].replace(0, np.nan))

        # 2. Median growth rates
        df['daily_growth'] = df['close'] / df['close'].shift(1)
        df['weekly_growth'] = df['close'] / df['close'].shift(7)
        df['monthly_growth'] = df['close'] / df['close'].shift(30)

        # 3. Rolling volatility
        df['vol_weekly'] = df['close'].pct_change().rolling(7).std()
        df['momentum_weekly'] = df['weekly_growth'] / df['vol_weekly']

        # 4. moving averages
        df['ma_20'] = df['close'].rolling(window=20).mean()
        df['ma_50'] = df['close'].rolling(window=50).mean()
        df['ma_100'] = df['close'].rolling(window=100).mean()

        # 5. Add metadata
        df['year'] = df['date'].dt.year
        df['quarter'] = df['date'].dt.quarter
        df["quarter_end"] = df["date"] + pd.offsets.QuarterEnd(0)
        
        return df

In [16]:
#df_crypto['Date'] = pd.to_datetime(df_crypto['Date'])

df_crypto_clean = (
    df_crypto
    .groupby('ticker', group_keys=False)
    .apply(get_price_features)
    .reset_index(drop=True)
)



C:\Users\HAOZHENG\AppData\Local\Temp\ipykernel_15672\2883351901.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_price_features)


In [18]:
df_crypto_clean.head(10).T

,0,1,2,3,4,5,6,7,8,9
date,2021-01-01 00:00:00,2021-01-01 00:00:00,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-02 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-03 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00
close,29331.69,728.91,1.8421,32178.33,774.56,1.7999,33000.05,978.28,2.1779,31988.71
volume,54182.925011,675114.09329,4421991.45,129993.873362,1352618.57668,5253819.62,120957.56675,2813603.88615,8318341.13,140899.88569
ticker,BTC/USDT,ETH/USDT,SOL/USDT,BTC/USDT,ETH/USDT,SOL/USDT,BTC/USDT,ETH/USDT,SOL/USDT,BTC/USDT
ln_volume,10.900121,13.422637,15.302101,11.775243,14.117553,15.474466,11.703195,14.849977,15.933973,11.855805
daily_growth,NaN,NaN,NaN,1.09705,1.062628,0.977091,1.025536,1.263014,1.210012,0.969353
weekly_growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monthly_growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vol_weekly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
momentum_weekly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_stock = pd.read_csv('CryptoEquities_OHLCV.csv', parse_dates=['Date'])

In [20]:
df_stock = df_stock[df_stock['Ticker'].apply(lambda x: x in ['MSTR', 'COIN', 'RIOT', 'MARA', 'CLSK', 'WULF'])].reset_index(drop=True)

In [56]:
#df_stock['Date'] = pd.to_datetime(df_stock['Date'])

df_stock_clean = (
    df_stock
    .groupby('Ticker', group_keys=False)
    .apply(get_price_features)
    .reset_index(drop=True)
)

C:\Users\HAOZHENG\AppData\Local\Temp\ipykernel_15672\3609911749.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_price_features)


In [ ]:
# FUTURE Growth: for binary models
df_stock_clean['positive_weekly_growth'] = np.where(df_stock_clean['weekly_growth'] > 1, 1, 0)

In [58]:
df_stock_clean['positive_weekly_growth'].value_counts()

positive_weekly_growth
0    3606
1    3344
Name: count, dtype: int64

In [59]:
df_stock_clean.head(10).T

,0,1,2,3,4,5,6,7,8,9
date,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-06 00:00:00,2021-01-07 00:00:00,2021-01-08 00:00:00,2021-01-11 00:00:00,2021-01-12 00:00:00,2021-01-13 00:00:00,2021-01-14 00:00:00,2021-01-15 00:00:00
close,42.522,42.863,48.055,53.575,53.164,49.549,51.424,51.926,63.1,57.807
volume,14679020,11006240,15360310,22844050,16453870,14856440,6717660,6174580,19117760,16088990
ticker,MSTR,MSTR,MSTR,MSTR,MSTR,MSTR,MSTR,MSTR,MSTR,MSTR
ln_volume,16.50193,16.213973,16.547297,16.944201,16.616071,16.513944,15.72025,15.635951,16.766128,16.593646
daily_growth,NaN,1.008019,1.12113,1.114868,0.992329,0.932003,1.037841,1.009762,1.215191,0.916117
weekly_growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.221156,1.472132,1.202934
monthly_growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vol_weekly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067675,0.095575,0.104843
momentum_weekly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.044539,15.402855,11.473716


In [60]:
macro_df.columns

Index(['quarter_end', 'gdp_us', 'gdp_us_yoy', 'gdp_us_qoq', 'cpi_us',
       'cpi_us_yoy', 'cpi_us_qoq', 'unemployment_us', 'unemployment_us_yoy',
       'unemployment_us_qoq', 'interest_us', 'interest_us_yoy',
       'interest_us_qoq', 'gdp_de', 'gdp_de_yoy', 'gdp_de_qoq', 'cpi_de',
       'cpi_de_yoy', 'cpi_de_qoq', 'interest_eu', 'interest_eu_yoy',
       'interest_eu_qoq', 'year', 'quarter'],
      dtype='object')

In [61]:
final_df = pd.DataFrame()

In [62]:
for crypto in df_crypto_clean['ticker'].unique():

    df_temp_crypto = df_crypto_clean[df_crypto_clean['ticker'] == crypto].copy()

    # Merge crypto and stock data on date
    df_merged = pd.merge(
            df_temp_crypto,
            df_stock_clean,
            on=['date', 'quarter_end', 'year', 'quarter'],
            suffixes=('_crypto', '_stock'),
            how = 'left'
    )
    final_df = pd.concat([final_df, df_merged], ignore_index=True)

In [63]:
final_df[final_df["ticker_crypto"] == "ETH/USDT"].head(10).T

,7485,7486,7487,7488,7489,7490,7491,7492,7493,7494
date,2021-01-01 00:00:00,2021-01-02 00:00:00,2021-01-03 00:00:00,2021-01-04 00:00:00,2021-01-04 00:00:00,2021-01-04 00:00:00,2021-01-04 00:00:00,2021-01-04 00:00:00,2021-01-05 00:00:00,2021-01-05 00:00:00
close_crypto,728.91,774.56,978.28,1041.43,1041.43,1041.43,1041.43,1041.43,1099.56,1099.56
volume_crypto,675114.09329,1352618.57668,2813603.88615,4245010.94004,4245010.94004,4245010.94004,4245010.94004,4245010.94004,2706995.67525,2706995.67525
ticker_crypto,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT,ETH/USDT
ln_volume_crypto,13.422637,14.117553,14.849977,15.261255,15.261255,15.261255,15.261255,15.261255,14.81135,14.81135
daily_growth_crypto,NaN,1.062628,1.263014,1.064552,1.064552,1.064552,1.064552,1.064552,1.055817,1.055817
weekly_growth_crypto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monthly_growth_crypto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vol_weekly_crypto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
momentum_weekly_crypto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
all_data = pd.merge(final_df, macro_df,
                    on = ['quarter_end', 'year', 'quarter'],
                    how = 'left')

In [68]:
all_data.to_csv('all_data_raw.csv', index=False)

In [69]:
all_data#.info()

,date,close_crypto,volume_crypto,ticker_crypto,ln_volume_crypto,daily_growth_crypto,weekly_growth_crypto,monthly_growth_crypto,vol_weekly_crypto,momentum_weekly_crypto,...,interest_us_qoq,gdp_de,gdp_de_yoy,gdp_de_qoq,cpi_de,cpi_de_yoy,cpi_de_qoq,interest_eu,interest_eu_yoy,interest_eu_qoq
0,2021-01-01,29331.69,5.418293e+04,BTC/USDT,10.900121,NaN,NaN,NaN,NaN,NaN,...,-0.125,743509.0,-0.006255,-0.006158,107.6424,0.024072,0.004922,0.147856,-3.686229,1.339151
1,2021-01-02,32178.33,1.299939e+05,BTC/USDT,11.775243,1.097050,NaN,NaN,NaN,NaN,...,-0.125,743509.0,-0.006255,-0.006158,107.6424,0.024072,0.004922,0.147856,-3.686229,1.339151
2,2021-01-03,33000.05,1.209576e+05,BTC/USDT,11.703195,1.025536,NaN,NaN,NaN,NaN,...,-0.125,743509.0,-0.006255,-0.006158,107.6424,0.024072,0.004922,0.147856,-3.686229,1.339151
3,2021-01-04,31988.71,1.408999e+05,BTC/USDT,11.855805,0.969353,NaN,NaN,NaN,NaN,...,-0.125,743509.0,-0.006255,-0.006158,107.6424,0.024072,0.004922,0.147856,-3.686229,1.339151
4,2021-01-04,31988.71,1.408999e+05,BTC/USDT,11.855805,0.969353,NaN,NaN,NaN,NaN,...,-0.125,743509.0,-0.006255,-0.006158,107.6424,0.024072,0.004922,0.147856,-3.686229,1.339151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22450,2025-08-29,205.08,8.607487e+06,SOL/USDT,15.968143,0.956396,1.021875,1.153625,0.053398,19.136770,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3.173563,-0.042311,0.028787
22451,2025-08-29,205.08,8.607487e+06,SOL/USDT,15.968143,0.956396,1.021875,1.153625,0.053398,19.136770,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3.173563,-0.042311,0.028787
22452,2025-08-30,202.70,3.894576e+06,SOL/USDT,15.175095,0.988395,0.992946,1.176983,0.053354,18.610669,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3.173563,-0.042311,0.028787
22453,2025-08-31,200.62,3.303224e+06,SOL/USDT,15.010410,0.989739,0.973978,1.232991,0.053323,18.265798,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3.173563,-0.042311,0.028787
